In [1]:
# !pip install imblearn
# !pip install numpy
# !pip install pandas
# !pip install tqdm
# !pip install scikit-learn
# !pip install torch
# !pip install flwr
# !pip install -U "flwr[simulation]"

In [2]:
# ==================== IMPORTS ====================
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score
import warnings
import os
import time
from scipy import stats
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cpu


In [3]:
# ==================== ADAPTIVE CONFIGURATION ====================
K = 6                    # Number of clients
local_epochs = 5         # Local epochs per client
T = 10                   # Rrounds 
batch_size = 128         # Batch size
latent_dim = 20
lr = 1e-4
kl_weight = 0.5

In [4]:
class AdaptiveCTVAE(nn.Module):
    def __init__(self, input_dim, latent_dim=15, dataset_num=1, use_bce=False):
        super(AdaptiveCTVAE, self).__init__()
        
        self.latent_dim = latent_dim
        self.dataset_num = dataset_num
        self.use_bce = use_bce
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.1),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(0.2),
            nn.Linear(32, latent_dim * 2)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.1),
            nn.Linear(32, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.1),
            nn.Linear(64, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )
        
        # Class embeddings با مقداردهی اولیه مناسب
        self.class_embed = nn.Embedding(2, latent_dim)
        nn.init.uniform_(self.class_embed.weight, -0.03, 0.03)
        self.eps = 1e-8
    
    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = h.chunk(2, dim=1)
        
        logvar = torch.clamp(logvar, -6, 6)
        
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x, y):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        
        # افزودن embedding کلاس با وزن مناسب
        class_emb = self.class_embed(y)
        z_cond = z + 0.1 * class_emb
        
        recon = self.decoder(z_cond)
        return recon, mu, logvar, z_cond
    
    def loss_fn(self, recon, x, mu, logvar, kl_weight=0.1):
        # Clamping برای جلوگیری از NaN
        recon = torch.clamp(recon, 1e-8, 1 - 1e-8)
        
        # انتخاب تابع loss بازسازی
        if self.use_bce:
            recon_loss = nn.BCELoss(reduction='mean')(recon, x)
        else:
            recon_loss = nn.MSELoss(reduction='mean')(recon, x)
        
        # محاسبه KL divergence با محافظت
        kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
        
        # وزن‌دهی بر اساس دیتاست
        total_loss = recon_loss + kl_weight * kl_loss
        
        # بررسی NaN
        if torch.isnan(total_loss).any():
            return torch.tensor(0.0, requires_grad=True, device=x.device)
        
        return total_loss, recon_loss, kl_loss

In [5]:
# ==================== IMPROVED DATA LOADING ====================
def load_iot_data_improved(dataset_number):
    """Load IoT dataset with intelligent preprocessing"""
    i = str(dataset_number)
    
    print(f"\nLoading Dataset {dataset_number}...")
    
    # لیست فایل‌ها
    benign_file = f"dataset/{i}/{i}.benign.csv"
    
    # بررسی وجود فایل
    if not os.path.exists(benign_file):
        raise FileNotFoundError(f"File not found: {benign_file}")
    
    # Load benign data
    benign_df = pd.read_csv(benign_file)
    
    # Load attack files
    attack_files = []
    
    # Gafgyt attacks
    gafgyt_types = ['combo', 'junk', 'scan', 'tcp', 'udp']
    for attack_type in gafgyt_types:
        attack_file = f"dataset/{i}/{i}.gafgyt.{attack_type}.csv"
        if os.path.exists(attack_file):
            attack_files.append(attack_file)
    
    # Mirai attacks (except for datasets 3 and 7)
    if dataset_number not in [3, 7]:
        mirai_types = ['scan', 'syn', 'udp', 'ack', 'udpplain']
        for attack_type in mirai_types:
            attack_file = f"dataset/{i}/{i}.mirai.{attack_type}.csv"
            if os.path.exists(attack_file):
                attack_files.append(attack_file)
    
    # Process benign data
    benign_df = benign_df.dropna()
    benign_train, benign_test = train_test_split(benign_df, test_size=0.3, random_state=42)
    
    # Load and process attack data
    attack_dfs = []
    for attack_file in attack_files:
        try:
            df = pd.read_csv(attack_file).dropna()
            if len(df) > 0:
                attack_dfs.append(df)
        except Exception as e:
            print(f"  ✗ Error loading {attack_file}: {e}")
    
    if attack_dfs:
        attacks = pd.concat(attack_dfs, ignore_index=True)
    else:
        attacks = pd.DataFrame()
        print(f"  No attack files found for dataset {dataset_number}")
    
    # ===== STRATEGIC DATA SPLITTING =====
    X_train = benign_train.drop(columns=['label'], errors='ignore').values
    y_train = np.zeros(len(benign_train))
    
    if len(attacks) > 0:
        n_attack_train = min(100000, len(attacks))
        attack_train = attacks.sample(n=n_attack_train, random_state=42)
        X_attack = attack_train.drop(columns=['label'], errors='ignore').values
        
        X_train = np.concatenate([X_train, X_attack])
        y_train = np.concatenate([y_train, np.ones(len(attack_train))])
    
    # ایجاد داده تست
    X_test = benign_test.drop(columns=['label'], errors='ignore').values
    y_test = np.zeros(len(benign_test))
    
    if len(attacks) > 0:
        max_attack_test = 20000
        
        if 'attack_train' in locals():
            remaining_attacks = attacks.drop(attack_train.index)
        else:
            remaining_attacks = attacks
        
        if len(remaining_attacks) > max_attack_test:
            attack_test = remaining_attacks.sample(n=max_attack_test, random_state=42)
        else:
            attack_test = remaining_attacks
        
        X_attack_test = attack_test.drop(columns=['label'], errors='ignore').values
        
        X_test = np.concatenate([X_test, X_attack_test])
        y_test = np.concatenate([y_test, np.ones(len(attack_test))])
    
    # ===== INTELLIGENT PREPROCESSING =====
    print(f"\nPreprocessing data...")
    
    scaler = MinMaxScaler(feature_range=(0.1, 0.9))
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    input_dim = X_train.shape[1]
    
    # Convert to tensors
    X_train_t = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_t = torch.tensor(y_train, dtype=torch.long).to(device)
    X_test_t = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_t = torch.tensor(y_test, dtype=torch.long).to(device)
    
    return X_train_t, y_train_t, X_test_t, y_test_t, input_dim

In [6]:
# ==================== FEDERATED CLIENT ====================
class FLClient:
    def __init__(self, cid, X, y, input_dim, latent_dim, dataset_num, use_bce=False):
        self.cid = cid
        self.X = X
        self.y = y
        self.dataset_num = dataset_num
        
        # مدل با تنظیمات ویژه
        self.model = AdaptiveCTVAE(input_dim, latent_dim, dataset_num, use_bce).to(device)
        
        # تنظیم optimizer
        self.optimizer = optim.AdamW(self.model.parameters(), lr=2e-4, weight_decay=5e-6)
        
        # ایجاد DataLoader
        dataset = TensorDataset(X, y)
        self.loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        
        # ثبت loss
        self.train_losses = []
    
    def train_local(self, global_state_dict, kl_weight=0.1):
        """Train locally and return update"""
        # بارگذاری مدل سرور
        self.model.load_state_dict(global_state_dict)
        self.model.train()
        
        total_loss = 0
        recon_loss_total = 0
        kl_loss_total = 0
        
        for epoch in range(local_epochs):
            epoch_loss = 0
            epoch_recon = 0
            epoch_kl = 0
            
            for batch_x, batch_y in self.loader:
                self.optimizer.zero_grad()
                
                recon, mu, logvar, _ = self.model(batch_x, batch_y)
                loss, recon_loss, kl_loss = self.model.loss_fn(recon, batch_x, mu, logvar, kl_weight)
                
                # بررسی NaN
                if torch.isnan(loss).any() or loss.item() > 1000:
                    print(f"Client {self.cid}: Invalid loss {loss.item():.4f}, skipping")
                    continue
                
                loss.backward()
                
                # Gradient clipping محکم
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=0.5)
                
                # بررسی gradient
                valid_grad = True
                for param in self.model.parameters():
                    if param.grad is not None and (torch.isnan(param.grad).any() or torch.isinf(param.grad).any()):
                        valid_grad = False
                        break
                
                if valid_grad:
                    self.optimizer.step()
                    epoch_loss += loss.item()
                    epoch_recon += recon_loss.item()
                    epoch_kl += kl_loss.item()
            
            if len(self.loader) > 0:
                avg_epoch_loss = epoch_loss / len(self.loader)
                avg_recon = epoch_recon / len(self.loader)
                avg_kl = epoch_kl / len(self.loader)
                
                total_loss += avg_epoch_loss
                recon_loss_total += avg_recon
                kl_loss_total += avg_kl
                
                self.train_losses.append(avg_epoch_loss)
        
        # محاسبه update
        local_state = self.model.state_dict()
        update = {}
        for key in global_state_dict.keys():
            update[key] = local_state[key].float() - global_state_dict[key].float()
        
        avg_total_loss = total_loss / local_epochs if local_epochs > 0 else 0
        avg_recon_loss = recon_loss_total / local_epochs if local_epochs > 0 else 0
        avg_kl_loss = kl_loss_total / local_epochs if local_epochs > 0 else 0
        
        return update, avg_total_loss, avg_recon_loss, avg_kl_loss, len(self.X)

In [7]:
class FLServer:
    def __init__(self, input_dim, latent_dim, dataset_num, use_bce=False):
        self.global_model = AdaptiveCTVAE(input_dim, latent_dim, dataset_num, use_bce).to(device)
        self.clients = []
        self.dataset_num = dataset_num
        self.round_losses = []
        self.round_recon_losses = []
        self.round_kl_losses = []
        
        # مقداردهی اولیه هوشمند
        self._initialize_model()
    
    def _initialize_model(self):
        """Initialize model with smart pre-training"""
        lr_init = 1e-4
        
        temp_optimizer = optim.AdamW(self.global_model.parameters(), lr=lr_init, weight_decay=1e-5)
        
        # داده‌های dummy با توزیع مناسب
        dummy_X = torch.randn(200, self.global_model.encoder[0].in_features).to(device)
        dummy_X = torch.clamp(dummy_X, 0.1, 0.9)
        dummy_y = torch.randint(0, 2, (200,)).to(device)
        
        print("  Initializing model with pre-training...")
        for init_step in range(10):
            temp_optimizer.zero_grad()
            recon, mu, logvar, _ = self.global_model(dummy_X, dummy_y)
            loss, _, _ = self.global_model.loss_fn(recon, dummy_X, mu, logvar)
            
            if not torch.isnan(loss).any():
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.global_model.parameters(), max_norm=0.3)
                temp_optimizer.step()
            
            if init_step % 2 == 0:
                print(f"    Pre-train step {init_step+1}/10: loss={loss.item():.6f}")
    
    def add_client(self, client):
        self.clients.append(client)
    
    def train_round(self, round_num, kl_weight=0.1):
        print(f"  Round {round_num}: Training clients...")
        
        global_state = self.global_model.state_dict()
        
        # جمع‌آوری updates از کلاینت‌ها
        updates = []
        losses = []
        recon_losses = []
        kl_losses = []
        client_samples = []
        
        for i, client in enumerate(self.clients):
            try:
                update, loss, recon_loss, kl_loss, n_samples = client.train_local(
                    global_state, kl_weight
                )
                
                # بررسی کیفیت update
                update_valid = True
                update_norm = 0.0
                
                for key in update.keys():
                    if torch.isnan(update[key]).any() or torch.isinf(update[key]).any():
                        update_valid = False
                        break
                    update_norm += torch.norm(update[key]).item()
                
                if update_valid and loss < 100:  # loss معقول
                    updates.append(update)
                    losses.append(loss)
                    recon_losses.append(recon_loss)
                    kl_losses.append(kl_loss)
                    client_samples.append(n_samples)
                    
                else:
                    zero_update = {key: torch.zeros_like(global_state[key]) for key in global_state.keys()}
                    updates.append(zero_update)
                    losses.append(1.0)
                    recon_losses.append(0.5)
                    kl_losses.append(0.5)
                    client_samples.append(n_samples)
                    
            except Exception as e:
                print(f"    Client {i+1}: Error - {str(e)[:50]}")
                zero_update = {key: torch.zeros_like(global_state[key]) for key in global_state.keys()}
                updates.append(zero_update)
                losses.append(2.0)
                recon_losses.append(1.0)
                kl_losses.append(1.0)
                client_samples.append(1000)
        
        # Federated Averaging با وزن‌دهی
        total_samples = sum(client_samples)
        if total_samples == 0:
            total_samples = 1
        
        avg_update = {}
        for key in updates[0].keys():
            weighted_sum = torch.zeros_like(updates[0][key], dtype=torch.float32)
            for i in range(len(updates)):
                weight = client_samples[i] / total_samples
                weighted_sum += weight * updates[i][key]
            avg_update[key] = weighted_sum
        
        # اعمال update با momentum تطبیقی
        momentum = 0.1
        
        new_state = {}
        for key in global_state.keys():
            current = global_state[key].float()
            update = avg_update[key].float()
            
            # محدود کردن بزرگی update
            update = torch.clamp(update, -0.2, 0.2)
            
            # اعمال momentum
            new_state[key] = (current + momentum * current + (1 - momentum) * update).to(global_state[key].dtype)
        
        # بررسی state جدید
        state_valid = True
        for key in new_state.keys():
            if torch.isnan(new_state[key]).any() or torch.isinf(new_state[key]).any():
                state_valid = False
                print(f"    Invalid state for {key}, keeping old state")
                break
        
        if state_valid:
            self.global_model.load_state_dict(new_state)
        
        # محاسبه میانگین loss
        avg_loss = np.mean(losses) if losses else 1.0
        avg_recon = np.mean(recon_losses) if recon_losses else 0.5
        avg_kl = np.mean(kl_losses) if kl_losses else 0.5
        
        self.round_losses.append(avg_loss)
        self.round_recon_losses.append(avg_recon)
        self.round_kl_losses.append(avg_kl)
        
        return avg_loss, avg_recon, avg_kl

In [8]:
# ==================== ENHANCED EVALUATION ====================
def evaluate_model_enhanced(model, X_train, y_train, X_test, y_test, dataset_num):
    """Evaluate the trained model with enhanced metrics"""
    model.eval()
    
    print(f"\n{'='*60}")
    print("MODEL EVALUATION")
    print(f"{'='*60}")
    
    # تابع استخراج features
    def extract_features(X, y, batch_size=4096):
        features = []
        labels = []
        
        with torch.no_grad():
            for i in range(0, len(X), batch_size):
                end = min(i + batch_size, len(X))
                batch_x = X[i:end]
                batch_y = y[i:end]
                
                _, _, _, z = model(batch_x, batch_y)
                z_np = z.cpu().numpy()
                
                # پاکسازی NaN
                if np.isnan(z_np).any():
                    z_np = np.nan_to_num(z_np, nan=0.0)
                
                features.append(z_np)
                labels.append(batch_y.cpu().numpy())
        
        features_array = np.concatenate(features)
        labels_array = np.concatenate(labels)
        
        return features_array, labels_array
    
    # آموزش RandomForest پیشرفته
    print(" Training RandomForest classifier...")
    
    rf_params = {
        'n_estimators': 100,
        'max_depth': 15,
        'min_samples_split': 10,
        'min_samples_leaf': 5,
        'max_features': 'sqrt',
        'random_state': 42,
        'n_jobs': -1
    }
    
    rf = RandomForestClassifier(**rf_params)
    
    z_train, y_train_clean = extract_features(X_train, y_train)
    z_test, y_test_clean = extract_features(X_test, y_test)
    rf.fit(z_train, y_train_clean)

    # پیش‌بینی
    print("\n Making predictions...")
    y_pred = rf.predict(z_test)
    y_true = y_test_clean
    
    # محاسبه معیارها
    acc = accuracy_score(y_true, y_pred) * 100
    f1 = f1_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred) * 100
    recall = recall_score(y_true, y_pred) * 100
    
    # ماتریس درهم‌ریختگی
    cm = confusion_matrix(y_true, y_pred)
    
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        
        # محاسبه نرخ‌ها
        fpr = fp / (fp + tn) * 100 if (fp + tn) > 0 else 0
        fnr = fn / (fn + tp) * 100 if (fn + tp) > 0 else 0
        tpr = tp / (tp + fn) * 100 if (tp + fn) > 0 else 0
        tnr = tn / (tn + fp) * 100 if (tn + fp) > 0 else 0
    else:
        tn = fp = fn = tp = 0
        fpr = fnr = tpr = tnr = 0
    
    # نمایش نتایج
    print(f"\n{'='*60}")
    print("EVALUATION RESULTS")
    print(f"{'='*60}")
    
    return acc, f1, cm, precision, recall, fpr, fnr

In [9]:
# ==================== MAIN FEDERATED TRAINING ====================
def run_federated_ctvae_complete(dataset_number):
    """Run complete federated CTVAE training"""
    start_time = time.time()
       
    # 1. بارگذاری داده‌ها
    print(f"\n[1/3] Loading data...")
    try:
        X_train, y_train, X_test, y_test, input_dim = load_iot_data_improved(dataset_number)
    except Exception as e:
        print(f" Error loading data: {e}")
        return 0, 0, 0
    
    # 2. راه‌اندازی Federated Learning
    print(f"\n[2/3] Initializing Federated Learning...")
    
    # ایجاد سرور با تنظیمات ویژه
    server = FLServer(input_dim, latent_dim, dataset_number, False)
    
    # تقسیم داده بین کلاینت‌ها
    n_total_samples = min(40000, len(X_train))  # افزایش حجم داده
    
    y_np = y_train.cpu().numpy()
    benign_idx = np.where(y_np == 0)[0]
    attack_idx = np.where(y_np == 1)[0]
    
    # برای دیتاست ۶، استفاده از داده‌های بیشتر
    
    n_per_client = n_total_samples // K
    
    n_per_class_per_client = n_per_client // 2
    
    for k in range(K):
        # نمونه‌گیری متعادل
        if len(benign_idx) >= n_per_class_per_client and len(attack_idx) >= n_per_class_per_client:
            client_benign = np.random.choice(benign_idx, n_per_class_per_client, replace=False)
            client_attack = np.random.choice(attack_idx, n_per_class_per_client, replace=False)
            
            client_idx = np.concatenate([client_benign, client_attack])
            np.random.shuffle(client_idx)
            
            client_X = X_train[client_idx]
            client_y = y_train[client_idx]
            
            # ایجاد کلاینت
            client = FLClient(k, client_X, client_y, input_dim, 
                            latent_dim, dataset_number, False)
            server.add_client(client)
            
        else:
            client_idx = np.random.choice(len(X_train), min(n_per_client, len(X_train)), replace=False)
            client_X = X_train[client_idx]
            client_y = y_train[client_idx]
            
            client = FLClient(k, client_X, client_y, input_dim,
                            latent_dim, dataset_number, False)
            server.add_client(client)
    
    # 3. آموزش فدرال
    print(f"\n[3/3] Federated Training ({T} rounds)...")
    
    round_losses = []
    round_recon_losses = []
    round_kl_losses = []
    
    for round_num in range(1, T + 1):
        print(f"\n  Round {round_num}/{T}")
        
        # آموزش راند
        loss, recon_loss, kl_loss = server.train_round(round_num, kl_weight)
        round_losses.append(loss)
        round_recon_losses.append(recon_loss)
        round_kl_losses.append(kl_loss)
    
    print(f"\n Training completed!")
    print(f"   Final loss: {round_losses[-1]:.6f}")
    print(f"   Training time: {time.time() - start_time:.1f} seconds")
    
    # 4. ارزیابی نهایی
   
    acc, f1, cm, precision, recall, fpr, fnr = evaluate_model_enhanced(
        server.global_model, X_train, y_train, X_test, y_test, dataset_number
    )
    
    # مقایسه با baseline
    baseline_acc = {1: 93.0, 2: 95.0, 3: 93.9, 4: 91.8, 5: 93.1, 6: 94.5, 7: 96.6, 8: 100.0, 9: 98.9}
    baseline_f1 = {1: 90.9, 2: 93.5, 3: 92.2, 4: 89.5, 5: 91.1, 6: 93.3, 7: 95.5, 8: 100.0, 9: 99.0}
    
    acc_diff = acc - baseline_acc[dataset_number]
    f1_diff = f1 - baseline_f1[dataset_number]
    
    print(f"\nDataset: IoT-{dataset_number}")
    print(f"Number of Clients: {K}")
    print(f"Number of Epochs: {local_epochs}")
    print(f"Number of Rounds: {T}")
    print(f"Batch Size: {batch_size}")
    print(f"Latent Dim: {latent_dim}")
    print(f"KL Weight: {kl_weight}")

    print("\n--- CTVAE Results ---")
    print(f"Accuracy: {baseline_acc[dataset_number]:>11.1f}")
    print(f"F1-Score: {baseline_f1[dataset_number]:>11.1f}")

    print("\n--- F-CTVAE Results ---")
    print(f"Accuracy: {acc:>11.2f} ({acc_diff:+.2f})")
    print(f"F1-Score: {f1:>11.2f} ({f1_diff:+.2f})")
    print("\nConfusion Matrix:\n", cm)
    
    return acc, f1, acc_diff

In [10]:
dataset_num = 1         # from 1 to 9

print("\n" + "═"*70)
print("F-CTVAE: Federated Constrained & Transformed VAE")
print("═"*70)

try:
    acc, f1, acc_diff = run_federated_ctvae_complete(dataset_num)
        
except FileNotFoundError as e:
    print(f"\n Error: {e}")
except Exception as e:
    print(f"\n Unexpected Error: {e}")
    import traceback
    traceback.print_exc()


══════════════════════════════════════════════════════════════════════
F-CTVAE: Federated Constrained & Transformed VAE
══════════════════════════════════════════════════════════════════════

[1/3] Loading data...

Loading Dataset 1...

Preprocessing data...

[2/3] Initializing Federated Learning...
  Initializing model with pre-training...
    Pre-train step 1/10: loss=0.155191
    Pre-train step 3/10: loss=0.154255
    Pre-train step 5/10: loss=0.152844
    Pre-train step 7/10: loss=0.152597
    Pre-train step 9/10: loss=0.152248

[3/3] Federated Training (10 rounds)...

  Round 1/10
  Round 1: Training clients...

  Round 2/10
  Round 2: Training clients...

  Round 3/10
  Round 3: Training clients...

  Round 4/10
  Round 4: Training clients...

  Round 5/10
  Round 5: Training clients...

  Round 6/10
  Round 6: Training clients...

  Round 7/10
  Round 7: Training clients...

  Round 8/10
  Round 8: Training clients...

  Round 9/10
  Round 9: Training clients...

  Round 10/10
